In [1]:
# !pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install -U numpy
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --version 1.9 --apt-packages libomp5 libopenblas-dev


In [2]:
# import torch
# !export XLA_USE_BF16=1
# import torch_xla.core.xla_model as xm
# print(xm.xla_device())

In [3]:
def prepare_dataset():

    # PRE-PROCESS LISA TRAFFIC LIGHT DATASET
    import glob
    import os
    import shutil
    import csv

    # make directories for dataset
    try:
        os.mkdir('./dataset')
        print("Directory 'dataset' is created")
    except OSError:
        print("Directory 'dataset' already exists")

    try:
        os.mkdir('./dataset/images')
        print("Directory 'images' is created")
    except OSError:
        print("Directory 'images' already exists")

    try:
        os.mkdir('./dataset/labels')
        print("Directory 'labels' is created")
    except OSError:
        print("Directory 'labels' already exists")

    # # Copying all images to one folder
    train_path = ['../input/lisa-traffic-light-dataset/dayTrain/dayTrain/dayClip*',
                 '../input/lisa-traffic-light-dataset/nightTrain/nightTrain/nightClip*']
    for i in train_path:
        print(i)
        for a in glob.glob(i):
            for b in glob.glob(a+'/*'):
                for c in glob.glob(b+'/*'):
                    shutil.copy(c, os.getcwd()+'/dataset/images/')

    for i in glob.glob('../input/lisa-traffic-light-dataset/*Sequence*'):
        print(i)
        for j in glob.glob(i+'/*'):
            for k in glob.glob(j+'/*'):
                for l in glob.glob(k+'/*'):
                    shutil.copy(l, os.getcwd()+'/dataset/images/')

    # Copying label files to labels folder and writing to one file
    with open('./dataset/trainLabels.csv', 'w', newline="") as f:
        for i in glob.glob('../input/lisa-traffic-light-dataset/Annotations/Annotations/*Train'):
            for j in glob.glob(i+'/*'):
                for k in glob.glob(j+'/*BOX.csv'):
                    file = open(k)
                    lines = file.readlines()[1:]    
                    for line in lines:
                        writer = csv.writer(f)
                        # write the data
                        writer.writerow([line[:-1]])

    with open('./dataset/testLabels.csv', 'w', newline="") as f:
        for i in glob.glob('../input/lisa-traffic-light-dataset/Annotations/Annotations/*Sequence*'):
            for j in glob.glob(i+'/*BOX.csv'):
                file = open(j)
                lines = file.readlines()[1:]    
                for line in lines:
                    writer = csv.writer(f)
                    # write the data
                    writer.writerow([line[:-1]])

    ## CREATING A LABEL MAP FOR CLASSES
    classes = {}
    file = open('./dataset/trainLabels.csv')
    for line in file.readlines():
        temp = line.split(';')
        try:
            classes[temp[1]] += 1
        except KeyError:
            classes[temp[1]] = 1
    file.close()

    file = open('./dataset/testLabels.csv')
    for line in file.readlines():
        temp = line.split(';')
        try:
            classes[temp[1]] += 1
        except KeyError:
            classes[temp[1]] = 1
    file.close()

    counter = 0
    label_map = {}
    for i in classes:
        label_map[i] = counter
        counter += 1

    label_map['goForward'] = 0
    print(label_map)

    # WRITING ALL DATA TO RIGHT FORMAT
    def get_line(line):
        temp = line.split(";")
        image_name = temp[0].split("/")[1]

        # Create the data line
        class_label = str(label_map[temp[1]])
        x = str( (int(temp[2]) + int(temp[4]))*0.5 / 1280)
        y = str( (int(temp[3]) + int(temp[5]))*0.5 / 960)
        w = str( (int(temp[4]) - int(temp[2])) /1280)
        h = str( (int(temp[5]) - int(temp[3])) /960)

        data = class_label + ' '
        data += x + ' '
        data += y + ' '
        data += w + ' '
        data += h

        return image_name, data    


    # ------------- FOR TRAIN DATA -------------------#

    file = open('./dataset/trainLabels.csv')
    annotations = {}
    for line in file.readlines():
        image_name, data = get_line(line)
        try:
            annotations[image_name]
            annotations[image_name].append(data)
        except KeyError:
            annotations[image_name] = [data]
    file.close()


    with open('./dataset/train.csv', 'w', newline="") as f1:

        for i in annotations:
            # Creating images and its text file associations
            text_file = i.split('.')[0] + ".txt"
            string = str(i) + str(",")+ str(text_file)
            writer1 = csv.writer(f1, quoting=csv.QUOTE_NONE, escapechar='', delimiter=" ")
            writer1.writerow([string])

            # Creating text files containing class and bounding boxes
            with open('./dataset/labels/'+text_file, 'w', newline="") as f2:
                for text_data in annotations[i]:
                    writer2 = csv.writer(f2)
                    writer2.writerow([text_data])


    # ------------- FOR TEST DATA -------------------#

    file = open('./dataset/testLabels.csv')
    annotations = {}
    for line in file.readlines():
        image_name, data = get_line(line)
        try:
            annotations[image_name]
            annotations[image_name].append(data)
        except KeyError:
            annotations[image_name] = [data]
    file.close()


    with open('./dataset/test.csv', 'w', newline="") as f1:

        for i in annotations:
            # Creating images and its text file associations
            text_file = i.split('.')[0] + ".txt"
            string = str(i) + str(",")+ str(text_file)
            writer1 = csv.writer(f1, quoting=csv.QUOTE_NONE, escapechar='', delimiter=" ")
            writer1.writerow([string])

            # Creating text files containing class and bounding boxes
            with open('./dataset/labels/'+text_file, 'w', newline="") as f2:
                for text_data in annotations[i]:
                    writer2 = csv.writer(f2)
                    writer2.writerow([text_data])

In [4]:
## Architecture

import torch
import torch.nn as nn
import torch.nn.functional as F

class architecture(nn.Module):
    def __init__(self, S, B, C):
        # S= No.of boxes, B=No.of prediction boxes, C=No.of classes
        super(architecture, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3,64,kernel_size=7,stride=2,padding=3),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(64,192,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(192),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(192,128,kernel_size=1,stride=1,padding=0),
            nn.Conv2d(128,256,kernel_size=3,stride=1,padding=1),
            nn.Conv2d(256,256,kernel_size=1,stride=1,padding=0),
            nn.Conv2d(256,512,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(512,256,kernel_size=1,stride=1,padding=0),
            nn.Conv2d(256,512,kernel_size=3,stride=1,padding=1),
            nn.Conv2d(512,256,kernel_size=1,stride=1,padding=0),
            nn.Conv2d(256,512,kernel_size=3,stride=1,padding=1),
            nn.Conv2d(512,256,kernel_size=1,stride=1,padding=0),
            nn.Conv2d(256,512,kernel_size=3,stride=1,padding=1),
            nn.Conv2d(512,256,kernel_size=1,stride=1,padding=0),
            nn.Conv2d(256,512,kernel_size=3,stride=1,padding=1),
            nn.Conv2d(512,512,kernel_size=1,stride=1,padding=0),
            nn.Conv2d(512,1024,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(1024, 512, kernel_size=1, stride=1, padding=0),
            nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1),
            nn.Conv2d(1024, 512, kernel_size=1, stride=1, padding=0),
            nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1),
            nn.Conv2d(1024, 1024, kernel_size=3, stride=1, padding=1),
            nn.Conv2d(1024, 1024, kernel_size=3, stride=2, padding=1),
            nn.Conv2d(1024, 1024, kernel_size=3, stride=1, padding=1),
            nn.Conv2d(1024, 1024, kernel_size=3, stride=1, padding=1),
            
            nn.Flatten(),
            nn.Linear(50176, 4096),
            nn.Dropout(0.0),
            nn.LeakyReLU(0.1),
            nn.Linear(4096, S * S * (C + B * 5))
        )
        
    def forward(self, x):
        return self.model(x)

In [5]:
## getDataset
from PIL import Image
import torch
import pandas as pd
import os

class load_ds(torch.utils.data.Dataset):
    def __init__(self, csv, images_dir, label_dir, S, B, C, transform=None):
        self.associations = pd.read_csv(csv)
        self.img_dir      = images_dir
        self.lbl_dir      = label_dir
        self.transform    = transform
        self.S            = S
        self.B            = B
        self.C            = C
        
    def __len__(self):
        return len(self.associations)
        
    def __getitem__(self, index):
        img_path   = os.path.join(self.img_dir, self.associations.iloc[index, 0])
        label_path = os.path.join(self.lbl_dir, self.associations.iloc[index, 1])
        bboxes     = []
        
        with open(label_path) as x:
            for i in x.readlines():
                class_label, x, y, w, h = [float(x) if float(x) != int(float(x)) else int(x)
                                            for x in i.replace("\n", "").split()]
                
                bboxes.append([class_label, x, y, w, h])
                
#         print(bboxes)
        
        img = Image.open(img_path)
        if self.transform:
            for i in self.transform:
                img, bboxes = i(img), bboxes
                
        
        labels = torch.zeros((self.S, self.S, self.C + self.B*5))
        for box in bboxes:
            class_label, x, y, w, h = box
            class_label = int(class_label)

            # i,j represents the cell row and cell column
            i, j = int(self.S * y), int(self.S * x)
            x_cell, y_cell = self.S * x - j, self.S * y - i
            w_cell, h_cell = (w*self.S, h*self.S)
            
            
            if labels[i, j, self.C] == 0:
                # Set 1 if there exists an object
                labels[i, j, self.C] = 1

                # Box coordinates
                box_coords = torch.tensor([x_cell, y_cell, w_cell, h_cell])
                labels[i, j, self.C+1:self.C+5] = box_coords

                # one hot encoding for class label
                labels[i, j, class_label] = 1

        return img, labels

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from collections import Counter

def iou(x, y, box_format):

        if box_format == "box":
            box1_x1 = x[..., 0]
            box1_y1 = x[..., 1]
            box1_x2 = x[..., 2]
            box1_y2 = x[..., 3]
            box2_x1 = y[..., 0]
            box2_y1 = y[..., 1]
            box2_x1 = y[..., 2]
            box2_y1 = y[..., 3]
        elif box_format == "midpoint":
            box1_x1 = x[..., 0:1] - x[..., 2:3] / 2
            box1_y1 = x[..., 1:2] - x[..., 3:4] / 2
            box1_x2 = x[..., 0:1] + x[..., 2:3] / 2
            box1_y2 = x[..., 1:2] + x[..., 3:4] / 2
            box2_x1 = y[..., 0:1] - y[..., 2:3] / 2
            box2_y1 = y[..., 1:2] - y[..., 3:4] / 2
            box2_x2 = y[..., 0:1] + y[..., 2:3] / 2
            box2_y2 = y[..., 1:2] + y[..., 3:4] / 2


        x1 = torch.max(box1_x1, box2_x1)
        y1 = torch.max(box1_y1, box2_y1)
        x2 = torch.min(box1_x2, box2_x2)
        y2 = torch.min(box1_y2, box2_y2)


        # .clamp(0) is for the case when they do not intersect
        intersection = (x2 - x1).clamp(0) * (y2 - y1).clamp(0)

        box1_area = abs((box1_x2 - box1_x1) * (box1_y2 - box1_y1))
        box2_area = abs((box2_x2 - box2_x1) * (box2_y2 - box2_y1))

        return intersection / (box1_area + box2_area - intersection + 1e-6)
    

def get_map(pred_boxes, true_boxes, iou_threshold, box_format, num_classes):

    # list storing all AP for respective classes
    average_precisions = []

    # used for numerical stability later on
    epsilon = 1e-6

    for c in range(num_classes):
        detections = []
        ground_truths = []

        # Go through all predictions and targets,
        # and only add the ones that belong to the
        # current class c
        for detection in pred_boxes:
            if detection[1] == c:
                detections.append(detection)

        for true_box in true_boxes:
            if true_box[1] == c:
                ground_truths.append(true_box)

        # find the amount of bboxes for each training example
        # Counter here finds how many ground truth bboxes we get
        # for each training example, so let's say img 0 has 3,
        # img 1 has 5 then we will obtain a dictionary with:
        # amount_bboxes = {0:3, 1:5}
        amount_bboxes = Counter([gt[0] for gt in ground_truths])

        # We then go through each key, val in this dictionary
        # and convert to the following (w.r.t same example):
        # ammount_bboxes = {0:torch.tensor[0,0,0], 1:torch.tensor[0,0,0,0,0]}
        for key, val in amount_bboxes.items():
            amount_bboxes[key] = torch.zeros(val)

        # sort by box probabilities which is index 2
        detections.sort(key=lambda x: x[2], reverse=True)
        TP = torch.zeros((len(detections)))
        FP = torch.zeros((len(detections)))
        total_true_bboxes = len(ground_truths)
        
        # If none exists for this class then we can safely skip
        if total_true_bboxes == 0:
            continue

        for detection_idx, detection in enumerate(detections):
            # Only take out the ground_truths that have the same
            # training idx as detection
            ground_truth_img = [
                bbox for bbox in ground_truths if bbox[0] == detection[0]
            ]

            num_gts = len(ground_truth_img)
            best_iou = 0

            for idx, gt in enumerate(ground_truth_img):
                iou_temp = iou(
                    torch.tensor(detection[3:]),
                    torch.tensor(gt[3:]),
                    box_format=box_format,
                )

                if iou_temp > best_iou:
                    best_iou = iou_temp
                    best_gt_idx = idx

            if best_iou > iou_threshold:
                # only detect ground truth detection once
                if amount_bboxes[detection[0]][best_gt_idx] == 0:
                    # true positive and add this bounding box to seen
                    TP[detection_idx] = 1
                    amount_bboxes[detection[0]][best_gt_idx] = 1
                else:
                    FP[detection_idx] = 1

            # if IOU is lower then the detection is a false positive
            else:
                FP[detection_idx] = 1

        TP_cumsum = torch.cumsum(TP, dim=0)
        FP_cumsum = torch.cumsum(FP, dim=0)
        recalls = TP_cumsum / (total_true_bboxes + epsilon)
        precisions = torch.divide(TP_cumsum, (TP_cumsum + FP_cumsum + epsilon))
        precisions = torch.cat((torch.tensor([1]), precisions))
        recalls = torch.cat((torch.tensor([0]), recalls))
        
        # torch.trapz for numerical integration
        average_precisions.append(torch.trapz(precisions, recalls))

    return sum(average_precisions) / len(average_precisions)

In [7]:
def get_bboxes(loader, model, iou_threshold, threshold,
               device, S, C, pred_format="cells", box_format="midpoint"):
    
    all_pred_boxes = []
    all_true_boxes = []

    # make sure model is in eval before get bboxes
    model.eval()
    train_idx = 0

    for batch_idx, (x, labels) in enumerate(loader):
        
#         print(f"get_bboxes batch idx: {batch_idx}")
        
        x      = x.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            predictions = model(x)

        batch_size  = x.shape[0]
        true_bboxes = cellboxes_to_boxes(labels, S, C)
        pred_bboxes = cellboxes_to_boxes(predictions, S, C)
        
#         print(f"true_bboxes length: {len(true_bboxes)}", f"pred_bboxes length: {len(pred_bboxes)}" )

        for idx in range(batch_size):
            nms_boxes = nms(
                pred_bboxes[idx],
                iou_threshold=iou_threshold,
                threshold=threshold,
                box_format=box_format)
            
#             print(f"Length of nms_boxes: {len(nms_boxes)}" )
            
            for nms_box in nms_boxes:
                all_pred_boxes.append([train_idx] + nms_box)

            for box in true_bboxes[idx]:
                # many will get converted to 0 pred
                if box[1] > threshold:
                    all_true_boxes.append([train_idx] + box)

            train_idx += 1
        
        break
        
    model.train()
    
    return all_pred_boxes, all_true_boxes


def nms(bboxes, iou_threshold, threshold, box_format="box"):

    assert type(bboxes) == list

    bboxes = [box for box in bboxes if box[1] > threshold]
    bboxes = sorted(bboxes, key=lambda x: x[1], reverse=True)
    bboxes_after_nms = []
    
#     print(f"Length of nms bboxes: {len(bboxes)}" )
    
    while bboxes:
        chosen_box = bboxes.pop(0)
        
        bboxes = [box for box in bboxes if box[0] != chosen_box[0]
                or iou(torch.tensor(chosen_box[2:]), torch.tensor(box[2:]), box_format=box_format) < iou_threshold]

        bboxes_after_nms.append(chosen_box)
        
    return bboxes_after_nms


def cellboxes_to_boxes(out, S, C):
    converted_pred = convert_cellboxes(out, S, C).reshape(out.shape[0], S * S, -1)
    converted_pred[..., 0] = converted_pred[..., 0].long()
    all_bboxes = []

    for ex_idx in range(out.shape[0]):
        bboxes = []

        for bbox_idx in range(S * S):
            bboxes.append([x.item() for x in converted_pred[ex_idx, bbox_idx, :]])
        all_bboxes.append(bboxes)

    return all_bboxes


def convert_cellboxes(predictions, S, C):

    predictions = predictions.to("cpu")
    batch_size  = predictions.shape[0]
    predictions = predictions.reshape(batch_size, S, S, C + B*5)
    bboxes1     = predictions[..., C+1:C+5]
    bboxes2     = predictions[..., C+6:C+10]
    scores      = torch.cat((predictions[..., C].unsqueeze(0), predictions[..., C+5].unsqueeze(0)), dim=0)
    
    best_box     = scores.argmax(0).unsqueeze(-1)
    best_boxes   = bboxes1 * (1 - best_box) + best_box * bboxes2
    cell_indices = torch.arange(S).repeat(batch_size, S, 1).unsqueeze(-1)
    
    x   = 1 / S * (best_boxes[..., :1] + cell_indices)
    y   = 1 / S * (best_boxes[..., 1:2] + cell_indices.permute(0, 2, 1, 3))
    w_y = 1 / S * best_boxes[..., 2:4]
    
    converted_bboxes = torch.cat((x, y, w_y), dim=-1)
    predicted_class  = predictions[..., :C].argmax(-1).unsqueeze(-1)
    best_confidence  = torch.max(predictions[..., C], predictions[..., C+5]).unsqueeze(-1)
    converted_preds  = torch.cat((predicted_class, best_confidence, converted_bboxes), dim=-1)
    
    return converted_preds

In [8]:
## LOSS
import torch
import torch.nn as nn
from torch import sqrt

class current_loss(nn.Module):
    def __init__(self, S, B, C, box_type):
        super(current_loss, self).__init__()
        self.S     = S
        self.B     = B
        self.C     = C
        self.btype = box_type
        self.mse   = nn.MSELoss(reduction="sum")
        
        self.lambda_noobj = 0.5
        self.lambda_coord = 5
        
    def forward(self, predictions, target):

        predictions = predictions.reshape(-1, self.S, self.S, self.C + self.B * 5)

        # Calculate IoU for the two predicted bounding boxes
        iou_b1 = iou(predictions[..., self.C+1:self.C+5], target[..., self.C+1:self.C+5], "midpoint")
        iou_b2 = iou(predictions[..., self.C+6:self.C+10], target[..., self.C+1:self.C+5], "midpoint")
        ious = torch.cat([iou_b1.unsqueeze(0), iou_b2.unsqueeze(0)], dim=0)

        # Take the box with highest IoU out of the two prediction
        iou_maxes, bestbox = torch.max(ious, dim=0)
        exists_box = target[..., self.C+1].unsqueeze(3)  # in paper this is Iobj_i

        # Loss for box coordinates and dimensions

        box_predictions = exists_box * (
            (
                bestbox * predictions[..., self.C+6:self.C+10]
                + (1 - bestbox) * predictions[..., self.C+1:self.C+5]
            )
        )

        box_targets = exists_box * target[..., self.C+1:self.C+5]

        # Take sqrt of width, height of boxes to ensure that
        box_predictions[..., 2:4] = torch.sign(box_predictions[..., 2:4]) * torch.sqrt(
            torch.abs(box_predictions[..., 2:4] + 1e-6)
        )
        box_targets[..., 2:4] = torch.sqrt(box_targets[..., 2:4])

        box_loss = self.mse(
            torch.flatten(box_predictions, end_dim=-2),
            torch.flatten(box_targets, end_dim=-2),
        )

        # Loss for existance of object

        # pred_box is the confidence score for the bbox with highest IoU
        pred_box = (
            bestbox * predictions[..., self.C+5:self.C+6] + (1 - bestbox) * predictions[..., self.C:self.C+1]
        )

        object_loss = self.mse(
            torch.flatten(exists_box * pred_box),
            torch.flatten(exists_box * target[..., self.C:self.C+1]),
        )

        # Loss for non existance of object

        no_object_loss = self.mse(
            torch.flatten((1 - exists_box) * predictions[..., self.C:self.C+1], start_dim=1),
            torch.flatten((1 - exists_box) * target[..., self.C:self.C+1], start_dim=1),
        )

        no_object_loss += self.mse(
            torch.flatten((1 - exists_box) * predictions[..., self.C+5:self.C+6], start_dim=1),
            torch.flatten((1 - exists_box) * target[..., self.C:self.C+1], start_dim=1)
        )

        #Class loss
        
        class_loss = self.mse(
            torch.flatten(exists_box * predictions[..., :self.C], end_dim=-2,),
            torch.flatten(exists_box * target[..., :self.C], end_dim=-2,),
        )

        loss = (
            self.lambda_coord * box_loss  # first two rows in paper
            + object_loss  # third row in paper
            + self.lambda_noobj * no_object_loss  # forth row
            + class_loss  # fifth row
        )

        return loss

In [9]:
## MAIN
import torch
import pandas
import matplotlib.pyplot as plt
import os
# import import_ipynb

os.getcwd()

from torchvision.datasets import ImageFolder
import torchvision.transforms as T

S = 16
B = 2
C = 6

# prepare_dataset()

# csv_directory, images_directory, labels_directory, transformation
train_dir  = './dataset/train.csv'
valid_dir  = './dataset/test.csv'
img_dir    = './dataset/images'
lbl_dir    = './dataset/labels'

to = [T.ToTensor(),T.Resize((448,448))]
train_ds  = load_ds(train_dir, img_dir, lbl_dir,
                   transform=to, S=S, B=B, C=C)
valid_ds  = load_ds(valid_dir, img_dir, lbl_dir,
                   transform=to, S=S, B=B, C=C)

from torch.utils.data.dataloader import DataLoader

batch_size  = 64
num_workers = 4
train_dl    = DataLoader(train_ds, batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
valid_dl    = DataLoader(valid_ds, batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)

# device = xm.xla_device()
device = 'cuda' if torch.cuda.is_available else 'cpu'
model  = architecture(S=S, B=B, C=C).to(device)
# model  = torch.load('../input/mymodel1/mymodel')
# model  = torch.load('mymodel')

from tqdm import tqdm
import torch.optim as optim
import warnings
warnings.filterwarnings('ignore')

myloss    = current_loss(S, B, C, 'midpoint')
optimizer = optim.Adam(model.parameters(), lr=0.00001, weight_decay=0)

In [10]:
# for idx, (x,y) in enumerate(train_dl):
#     x = x.to(device)
#     preds = model(x)
#     print(idx, x.shape, y.shape, preds.shape)
#     print(model.model)
#     break

In [ ]:
epochs     = 10
loss_trend = []
print("Using :", device)

for epoch in range(epochs):
    avg_loss  = []
    loop = tqdm(enumerate(train_dl), total=len(train_dl), leave=False)
    for batch_idx, (x, y) in loop:
        x = x.to(device)
        y = y.to(device)
        loss = myloss(model(x), y)
        avg_loss.append(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loop.set_description(f"Epoch: [{epoch+1}/{epochs}]")
        loop.set_postfix(loss=loss.item())
        
    loss_trend.append(sum(avg_loss)/len(avg_loss))
    
    pred_boxes, target_boxes = get_bboxes(valid_dl, model, iou_threshold=0.5, threshold=0.4, device=device, S=S, C=C)
    mAP = None
    try:
        mAP = get_map(pred_boxes, target_boxes, iou_threshold=0.5, box_format="midpoint", num_classes=C)
    except ZeroDivisionError:
        print("ZeroDivisionError")
        
    print(f"Epoch : {epoch+1}", f" Training Loss : {loss_trend[-1]}", f" Validation mAP : {mAP}")   


torch.save(model, 'mymodel')
print("Model saved!")


Using : cuda


Epoch : 1  Training Loss : 160.8836212158203  Validation mAP : 0.0


Epoch : 2  Training Loss : 78.28256225585938  Validation mAP : 0.0


Epoch : 3  Training Loss : 60.010650634765625  Validation mAP : 0.0


Epoch: [4/10]:  67%|██████▋   | 190/282 [05:51<03:22,  2.20s/it, loss=55.6]